In [301]:
# imports 
import numpy as np
import pandas as pd
import os
import subprocess
from IPython.display import HTML


In [302]:
# Load attribution and one-hot sequences and verify lengths

# Load DEV library and attributions
dev_library = pd.read_csv('Libraries/dev_modisco/dev_library.csv', index_col=0)
dev_attr = np.load('Libraries/dev_modisco/attr.npz')['arr_0']
dev_ohe = np.load('Libraries/dev_modisco/ohe.npz')['arr_0']

# Load HK library and attributions
hk_library = pd.read_csv('Libraries/hk_modisco/hk_library.csv', index_col=0)
hk_attr = np.load('Libraries/hk_modisco/attr.npz')['arr_0']
hk_ohe = np.load('Libraries/hk_modisco/ohe.npz')['arr_0']

# Print proof that they're the same length
print("DEV Library:")
print(f"  Library sequences: {len(dev_library)}")
print(f"  Attribution shape: {dev_attr.shape}")
print(f"  One-hot shape: {dev_ohe.shape}")
print(f"  Match: {len(dev_library) == dev_attr.shape[0] == dev_ohe.shape[0]}")

print("\nHK Library:")
print(f"  Library sequences: {len(hk_library)}")
print(f"  Attribution shape: {hk_attr.shape}")
print(f"  One-hot shape: {hk_ohe.shape}")
print(f"  Match: {len(hk_library) == hk_attr.shape[0] == hk_ohe.shape[0]}")

print(f"\nTotal sequences loaded:")
print(f"  DEV: {len(dev_library)} sequences")
print(f"  HK: {len(hk_library)} sequences")
print(f"  Total: {len(dev_library) + len(hk_library)} sequences")

DEV Library:
  Library sequences: 302
  Attribution shape: (302, 4, 249)
  One-hot shape: (302, 4, 249)
  Match: True

HK Library:
  Library sequences: 336
  Attribution shape: (336, 4, 249)
  One-hot shape: (336, 4, 249)
  Match: True

Total sequences loaded:
  DEV: 302 sequences
  HK: 336 sequences
  Total: 638 sequences


In [303]:
dev_library.head(10)


,ce_name,orientation,ce_sequence,endogenous_sequence,pseudo_index,ce_start,ce_end,log2_fc,p_value,dataset,EvoAug_predictions
0,"('Dref', 'GATAe', '+', '+', 5)",+,TATCGATAGATTGCGATAAC,TTACTTTATACTTGAGATAGTAGTCTATCGATAGATTGCGATAACCTAAAAAAAATACCACATTGCAGTATTTTGAAGCTTAGACAAGGAAAATGTTTCGCTTTCTTCTAACAAATTACTACAATTGTGAGGTCAAATTCACGCAGATCCCAGTTCTGCACTTCAAATCGCGAAGTATAGTTTTTTATATTTATTTTTTTTTGCCAGTAACGGGAAACACTTCAGACTCTATTATTTTGCTTCTTTTTT,1584661,25,44,5.321928,3.162613e-04,dev,1.731806
1,"('twi', 'twi', '+', '-', 6)",+,ACACATATATGTACATATGT,CAAGTGCCGCAGAAACATTTGCTGATAAAATGCAAGATTTCATAATGAGCTATTGTTCTTTGGCAACCGATACATGAGTCTGGAATGGAAATGTCACAATATCGTACTAAGTTCTTCTCTTTTTTTTTATAGGGGTAGAGCAATGCCCAGTAGATACACATATATGTACATATGTAAGTATGTGCGTATGTTTATGGCAAGCACCGACTACCGCGATAGATAAGAAACCTTCCAGTTCATTCAGTTTTT,846257,155,174,36.219281,4.635498e-09,dev,3.400490
2,"('Dref', 'GATAe', '+', '+', -5)",+,TATCGATAAG,ATTTAGCACGAGAAAACACAGATAATTGGCTATTCACAGCGACATATCACACTTTTGTTCTTTATTCGCACTCGATCGCTAGCCAACACTAGTAGTTCTCAGTGCCCTCGGCACTTTGCAACACGCCACCCGCACAGAACAGGTGCGGCGTTGCCAGACAAACGCCGAACACCGCAGAATGTAAACAAAGCACCAGTGCTGCATAGCAGAATTAGCCATTTATCGATAAGTTCTCTTGGCGCGCTTTTT,620728,220,229,4.574315,5.067241e-114,dev,2.671588
3,"('Dref', 'GATAe', '+', '+', -5)",+,TATCGATAAG,AAAAATAGTTTTAAATAATTTTATTGAAATAGAAGTTTCAAAAAATCCGCCCCTTGTGCCTAGTGTGACCGTATAATATACAACCGTATATACTGAAACACATGGCCGTGGCTATCGATAAGTCGATAACGAAATTTGAATTTCTTCCGTCTCTAAATATTTTGGTTGATCGGAACGCGCACGTGTTTTTGCGTTCAATTTTAATAATATTTGAAAATGGGATTTGGTAAACCTCGTGGCGGCGGTGGC,433870,112,121,4.574315,5.067241e-114,dev,3.646802
4,"('Dref', 'GATAe', '+', '+', -5)",+,TATCGATAAC,AAAACGAAAACGTAGAGCTACCGTGCTTGGGTTCGATACTTGTGCTTGGTGGCTATCGATAACAGGCTGAACATCACTAGCGATAGAAGGCGCTTGTGTGCATCAACCGTTTGCATGTGCGCTGCAAGAAAATATTTAATGATTGAATGATTTGTAGTAAATACTATATCAATAAAAATTAATAGGCAAAGTCAATGTTTATTTCAAAACATAATTAATAGTATTAAAACCTGCATAACAAAGTGTTTT,639485,53,62,4.574315,5.067241e-114,dev,2.333950
5,"('Dref', 'GATAe', '+', '+', -5)",+,TATCGATAAG,AAAACATTTGATCAAAAACACAAATAAGAAAAATTTCCCACCATTTCGTCCAATCTTTTCAACGAGCCACACCGGAAAGAAAGAACCAGACCGGAAAAGAATACGCGCTGGTGGTGGGTTTATATCGATAAGTTCACTTATCACAATTTCATGCAATACGGAATGATGAAATAGAACGAAAAATATTTGAGGCATTACAAAAAATTATAATATAATATTTAAGACTTTTTTTTTATAGCAAGGGTGGTT,710270,122,131,4.574315,5.067241e-114,dev,1.482395
6,"('Dref', 'GATAe', '+', '+', -5)",+,TATCGATAAC,AAAACGCAATATTCGTTTTTACATTCGCATTTGTTGCGAATAAATAAATGGACCTAAATATTATACCGAACGCGTTGAAGTGAAGTTGGCTGCGATTGGGAAAACGAAAACGTAGAGCTACCGTGCTTGGGTTCGATACTTGTGCTTGGTGGCTATCGATAACAGGCTGAACATCACTAGCGATAGAAGGCGCTTGTGTGCATCAACCGTTTGCATGTGCGCTGCAAGAAAATATTTAATGATTGAATG,139095,153,162,4.574315,5.067241e-114,dev,2.889539
7,"('Dref', 'GATAe', '+', '+', -5)",+,TATCGATAAG,GCGTAACTTTGAAAGACTTTCAGAAAAATATCAATTTTTGGTTTCCCACTCTTTTTCCTTTACTACAACGCGTTAGAGATGGGAAAACACACGGAATAACTATCACCGCGTATCGATAAGTGTTCGCTGCGAATGGCTGGCAGGGATGAGGAAGAGAAGCCAGCCTATCGGCGTATCGATAACACAACAATAATCCACGTACAAGTCGCGCATTTGGCACTGATTTTTATTAAAATGTAATTCAGTTTT,18536,110,119,4.574315,5.067241e-114,dev,4.725170
8,"('GATAe', 'Jra', '+', '+', 4)",+,AGATAAGCTAAGTGAATCATG,TGCCGTCAGATAAGCTAAGTGAATCATGCAATGAAGAGAGGCAATGTGTGGAGGCGGATATCACAAAGGAAGTGGCAGATCCAACGACCTCTAAGAATGAAGCTGCAGCATCGGTGGATGAATTACCAAGCCAGAGCCGGGAGATAAAAACGGCTGAAAACGCAAGCAAATCTGTAGCTGACAAAAAGGAAGACAACGAGGAAACCGTAAGTATACAAAATGTTATCAATTTGTGACAAGTTATCTTTT,276895,7,27,6.643856,3.851699e-10,dev,2.230574
9,"('Dref', 'Dref', '+', '-', 5)",+,TATCGATAGATTCAGTCGATA,GAGTTGCAGAGCTGCGAGGAATGATTAAAGCTGCAGGCCGCTTAGATAAGCGCAATGTGCGCTCTCCTCCCACTTTTATTAGCAATCGTCTGCGTTAATGTATTGTCATTTTGTAATAATTTATTAATTTTAATGTTTGAAATAAATATACTCTGTAAAAAGTGTGAACATAATATGTTAATCAAACCAGTTGGCAGGCCGCTCGCTGGCTGCTATCGATAGATTCAGTCGATATCGCCCGTGGCTTTT,847311,213,233,5.906891,2.398414e-11,dev,1.967761


In [304]:
# Create results directories
os.makedirs('Libraries/dev_modisco/results', exist_ok=True)
os.makedirs('Libraries/hk_modisco/results', exist_ok=True)

# Run modisco for DEV library with window size 249
print("Running modisco for DEV library...")
dev_cmd = [
    'modisco', 'motifs',
    '-s', 'Libraries/dev_modisco/ohe.npz',
    '-a', 'Libraries/dev_modisco/attr.npz', 
    '-n', '2000',
    '-w', '248',  # Set window size to 248 (same as sequence length)
    '-o', 'Libraries/dev_modisco/results/modisco_results.h5'
]

result_dev = subprocess.run(dev_cmd, capture_output=True, text=True)
print(f"DEV modisco exit code: {result_dev.returncode}")
if result_dev.stdout:
    print(f"DEV stdout: {result_dev.stdout}")
if result_dev.stderr:
    print(f"DEV stderr: {result_dev.stderr}")

# Run modisco for HK library with window size 249
print("\nRunning modisco for HK library...")
hk_cmd = [
    'modisco', 'motifs',
    '-s', 'Libraries/hk_modisco/ohe.npz',
    '-a', 'Libraries/hk_modisco/attr.npz',
    '-n', '2000',
    '-w', '248',  # Set window size to 248 (same as sequence length)
    '-o', 'Libraries/hk_modisco/results/modisco_results.h5'
]

result_hk = subprocess.run(hk_cmd, capture_output=True, text=True)
print(f"HK modisco exit code: {result_hk.returncode}")
if result_hk.stdout:
    print(f"HK stdout: {result_hk.stdout}")
if result_hk.stderr:
    print(f"HK stderr: {result_hk.stderr}")

print(f"\nModisco results saved to:")
print(f"  DEV: Libraries/dev_modisco/results/modisco_results.h5")
print(f"  HK: Libraries/hk_modisco/results/modisco_results.h5")

Running modisco for DEV library...
DEV modisco exit code: 0

Running modisco for HK library...
HK modisco exit code: 0

Modisco results saved to:
  DEV: Libraries/dev_modisco/results/modisco_results.h5
  HK: Libraries/hk_modisco/results/modisco_results.h5


In [305]:
import subprocess
import os
from IPython.display import HTML

# Run modisco report for DEV library
print("Running modisco report for DEV library...")
dev_report_cmd = [
    'modisco', 'report',
    '-i', 'Libraries/dev_modisco/results/modisco_results.h5',
    '-o', 'Libraries/dev_modisco/results/report/'
]

result_dev_report = subprocess.run(dev_report_cmd, capture_output=True, text=True)
print(f"DEV report exit code: {result_dev_report.returncode}")
if result_dev_report.stdout:
    print(f"DEV report stdout: {result_dev_report.stdout}")
if result_dev_report.stderr:
    print(f"DEV report stderr: {result_dev_report.stderr}")

# Run modisco report for HK library
print("\nRunning modisco report for HK library...")
hk_report_cmd = [
    'modisco', 'report',
    '-i', 'Libraries/hk_modisco/results/modisco_results.h5',
    '-o', 'Libraries/hk_modisco/results/report/'
]

result_hk_report = subprocess.run(hk_report_cmd, capture_output=True, text=True)
print(f"HK report exit code: {result_hk_report.returncode}")
if result_hk_report.stdout:
    print(f"HK report stdout: {result_hk_report.stdout}")
if result_hk_report.stderr:
    print(f"HK report stderr: {result_hk_report.stderr}")

# Check if reports were generated
dev_report_dir = 'Libraries/dev_modisco/results/report'
hk_report_dir = 'Libraries/hk_modisco/results/report'

print("\nDEV Modisco Report:")
if os.path.exists(dev_report_dir):
    print(f"Report directory exists: {dev_report_dir}")
else:
    print("Report directory not found")

print("\n" + "="*50 + "\n")

print("HK Modisco Report:")
if os.path.exists(hk_report_dir):
    print(f"Report directory exists: {hk_report_dir}")
else:
    print("Report directory not found")

Running modisco report for DEV library...
DEV report exit code: 0

Running modisco report for HK library...
HK report exit code: 0

DEV Modisco Report:
Report directory exists: Libraries/dev_modisco/results/report


HK Modisco Report:
Report directory exists: Libraries/hk_modisco/results/report


In [306]:
HTML('Libraries/dev_modisco/results/report/motifs.html')


pattern,num_seqlets,modisco_cwm_fwd,modisco_cwm_rev
pos_patterns.pattern_0,47,,
pos_patterns.pattern_1,47,,
pos_patterns.pattern_2,44,,


In [307]:
HTML('Libraries/hk_modisco/results/report/motifs.html')

pattern,num_seqlets,modisco_cwm_fwd,modisco_cwm_rev
pos_patterns.pattern_0,96,,


In [308]:
# Download JASPAR database for TOMTOM comparison
import subprocess
import os

# Create CEseek_data directory if it doesn't exist
os.makedirs('CEseek_data', exist_ok=True)

print("Downloading JASPAR database...")
download_cmd = [
    'wget', '--quiet',
    'https://jaspar.genereg.net/download/data/2022/CORE/JASPAR2022_CORE_vertebrates_non-redundant_pfms_meme.txt',
    '-O', 'CEseek_data/JASPAR2022_CORE_vertebrates_non-redundant_pfms_meme.txt'
]

result_download = subprocess.run(download_cmd, capture_output=True, text=True)
print(f"Download exit code: {result_download.returncode}")
if result_download.stderr:
    print(f"Download stderr: {result_download.stderr}")

# Check if file was downloaded
if os.path.exists('CEseek_data/JASPAR2022_CORE_vertebrates_non-redundant_pfms_meme.txt'):
    print("JASPAR database downloaded successfully")
else:
    print("Failed to download JASPAR database")

Download exit code: 0
JASPAR database downloaded successfully


In [309]:
# Run modisco report with TOMTOM comparison for DEV library
print("Running modisco report with TOMTOM for DEV library...")
dev_report_cmd = [
    'modisco', 'report',
    '-i', 'Libraries/dev_modisco/results/modisco_results.h5',
    '-o', 'Libraries/dev_modisco/results/report/',
    '-s', 'Libraries/dev_modisco/results/report/',
    '-m', 'CEseek_data/JASPAR2022_CORE_vertebrates_non-redundant_pfms_meme.txt'
]

result_dev_report = subprocess.run(dev_report_cmd, capture_output=True, text=True)
print(f"DEV report with TOMTOM exit code: {result_dev_report.returncode}")
if result_dev_report.stdout:
    print(f"DEV report stdout: {result_dev_report.stdout}")
if result_dev_report.stderr:
    print(f"DEV report stderr: {result_dev_report.stderr}")

# Run modisco report with TOMTOM comparison for HK library
print("\nRunning modisco report with TOMTOM for HK library...")
hk_report_cmd = [
    'modisco', 'report',
    '-i', 'Libraries/hk_modisco/results/modisco_results.h5',
    '-o', 'Libraries/hk_modisco/results/report/',
    '-s', 'Libraries/hk_modisco/results/report/',
    '-m', 'CEseek_data/JASPAR2022_CORE_vertebrates_non-redundant_pfms_meme.txt'
]

result_hk_report = subprocess.run(hk_report_cmd, capture_output=True, text=True)
print(f"HK report with TOMTOM exit code: {result_hk_report.returncode}")
if result_hk_report.stdout:
    print(f"HK report stdout: {result_hk_report.stdout}")
if result_hk_report.stderr:
    print(f"HK report stderr: {result_hk_report.stderr}")

Running modisco report with TOMTOM for DEV library...
DEV report with TOMTOM exit code: 0

Running modisco report with TOMTOM for HK library...
HK report with TOMTOM exit code: 0


In [310]:
# Display the updated reports with TOMTOM comparisons
from IPython.display import HTML

print("DEV Modisco Report with TOMTOM:")
HTML('Libraries/dev_modisco/results/report/motifs.html')

DEV Modisco Report with TOMTOM:


pattern,num_seqlets,modisco_cwm_fwd,modisco_cwm_rev,match0,qval0,match0_logo,match1,qval1,match1_logo,match2,qval2,match2_logo
pos_patterns.pattern_0,47,,,MA0754.2,1.000000,,MA0755.1,1.000000,,MA0660.1,1.000000,
pos_patterns.pattern_1,47,,,MA1141.1,0.000273,,MA1142.1,0.000273,,MA1988.1,0.000273,
pos_patterns.pattern_2,44,,,MA0036.3,0.325289,,MA0037.4,0.325289,,MA0482.2,0.325289,


In [311]:
print("HK Modisco Report with TOMTOM:")
HTML('Libraries/hk_modisco/results/report/motifs.html')

HK Modisco Report with TOMTOM:


pattern,num_seqlets,modisco_cwm_fwd,modisco_cwm_rev,match0,qval0,match0_logo,match1,qval1,match1_logo,match2,qval2,match2_logo
pos_patterns.pattern_0,96,,,MA0755.1,0.260965,,MA0756.2,0.283614,,MA1718.1,1.0,


In [320]:
# Create TOMTOM_motifs column using the same results as motifs.html
from bs4 import BeautifulSoup

def extract_tomtom_from_html(html_file):
    """Extract TOMTOM motif names from the HTML report."""
    with open(html_file, 'r') as f:
        html_content = f.read()
    
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find('table')
    
    tomtom_motifs = {}
    
    if table:
        rows = table.find_all('tr')[1:]  # Skip header row
        
        for row in rows:
            cells = row.find_all('td')
            if len(cells) >= 4:
                pattern_name = cells[0].text.strip()
                # TOMTOM matches are in cells 4, 7, 10 (every 3rd cell starting from 4)
                tomtom_matches = []
                for i in [4, 7, 10]:  # Specific cells with TOMTOM matches
                    if i < len(cells):
                        match_text = cells[i].text.strip()
                        if match_text and match_text != '-':
                            tomtom_matches.append(match_text)
                
                if tomtom_matches:
                    # Use the first (best) match
                    tomtom_motifs[pattern_name] = tomtom_matches[0]
                else:
                    tomtom_motifs[pattern_name] = pattern_name
    
    return tomtom_motifs

# Extract TOMTOM motif names from HTML reports
dev_tomtom_names = extract_tomtom_from_html('Libraries/dev_modisco/results/report/motifs.html')
hk_tomtom_names = extract_tomtom_from_html('Libraries/hk_modisco/results/report/motifs.html')

# Create simplified mapping (remove 'pos_patterns.' prefix)
dev_tomtom_simple = {}
for key, value in dev_tomtom_names.items():
    simple_key = key.replace('pos_patterns.', '')
    dev_tomtom_simple[simple_key] = value

hk_tomtom_simple = {}
for key, value in hk_tomtom_names.items():
    simple_key = key.replace('pos_patterns.', '')
    hk_tomtom_simple[simple_key] = value

# Create TOMTOM_motifs column using actual patterns found
import h5py

def create_tomtom_motifs_column(library_df, tomtom_mapping, modisco_h5_path):
    """Create TOMTOM_motifs column using patterns found in TOMTOM results."""
    motif_assignments = []
    
    # Get the patterns that were actually found
    available_patterns = list(tomtom_mapping.keys())
    
    # Load seqlets data from modisco results to get actual positions
    seq_to_patterns = {}
    
    with h5py.File(modisco_h5_path, 'r') as f:
        pos_patterns = f['pos_patterns']
        
        for pattern_name in available_patterns:
            if pattern_name in pos_patterns:
                pattern_group = pos_patterns[pattern_name]
                if 'seqlets' in pattern_group:
                    seqlets = pattern_group['seqlets']
                    if 'example_idx' in seqlets and 'start' in seqlets and 'end' in seqlets:
                        example_indices = seqlets['example_idx'][:]
                        start_positions = seqlets['start'][:]
                        end_positions = seqlets['end'][:]
                        
                        # Check for multiple hits to same sequence
                        seq_hit_counts = {}
                        for seq_idx in example_indices:
                            seq_hit_counts[seq_idx] = seq_hit_counts.get(seq_idx, 0) + 1
                        
                        multiple_hits = {seq: count for seq, count in seq_hit_counts.items() if count > 1}
                        if multiple_hits:
                            print(f"Sequences with multiple hits to {pattern_name}: {multiple_hits}")
                        
                        # Map each sequence to its patterns with actual positions
                        for i, seq_idx in enumerate(example_indices):
                            if seq_idx not in seq_to_patterns:
                                seq_to_patterns[seq_idx] = []
                            
                            start_pos = start_positions[i]
                            end_pos = end_positions[i]
                            # Check if this is reverse complement
                            is_revcomp = False
                            if 'is_revcomp' in seqlets:
                                is_revcomp = seqlets['is_revcomp'][i]
                            
                            direction = 'rev' if is_revcomp else 'fwd'
                            seq_to_patterns[seq_idx].append(f"{pattern_name}({start_pos}-{end_pos},{direction})")
    
    # Assign patterns to sequences
    for idx, row in library_df.iterrows():
        if idx in seq_to_patterns:
            # Join multiple patterns if a sequence has multiple matches
            patterns = '; '.join(seq_to_patterns[idx])
            motif_assignments.append(patterns)
        else:
            motif_assignments.append("No motifs found")
    
    return motif_assignments

# Create TOMTOM_motifs columns
dev_library['TOMTOM_motifs'] = create_tomtom_motifs_column(dev_library, dev_tomtom_simple, 'Libraries/dev_modisco/results/modisco_results.h5')
hk_library['TOMTOM_motifs'] = create_tomtom_motifs_column(hk_library, hk_tomtom_simple, 'Libraries/hk_modisco/results/modisco_results.h5')

# Update TOMTOM_motifs column with MA codes but keep positions
def replace_pattern_with_name(motif_string, tomtom_mapping):
    if motif_string == 'No motifs found':
        return 'No motifs found'
    
    # Handle multiple motifs separated by semicolon
    if ';' in motif_string:
        motifs = motif_string.split('; ')
        replaced_motifs = []
        for motif in motifs:
            if '(' in motif:
                pattern_name = motif.split('(')[0]
                position_info = motif.split('(')[1]
                if pattern_name in tomtom_mapping:
                    replaced_motifs.append(f"{tomtom_mapping[pattern_name]}({position_info}")
                else:
                    replaced_motifs.append(motif)
            else:
                replaced_motifs.append(motif)
        return '; '.join(replaced_motifs)
    else:
        # Single motif
        if '(' in motif_string:
            pattern_name = motif_string.split('(')[0]
            position_info = motif_string.split('(')[1]
            if pattern_name in tomtom_mapping:
                return f"{tomtom_mapping[pattern_name]}({position_info}"
            else:
                return motif_string
        else:
            return motif_string

# Update motif_names column with only the names (MA codes)
def extract_motif_names_only(motif_string, tomtom_mapping):
    if motif_string == 'No motifs found':
        return 'No motifs found'
    
    # Handle multiple motifs separated by semicolon
    if ';' in motif_string:
        motifs = motif_string.split('; ')
        motif_names = []
        for motif in motifs:
            if '(' in motif:
                pattern_name = motif.split('(')[0]
                if pattern_name in tomtom_mapping:
                    motif_names.append(tomtom_mapping[pattern_name])
                else:
                    motif_names.append(pattern_name)
            else:
                motif_names.append(motif)
        # Keep all motif names (including duplicates for multiple hits)
        return ', '.join(motif_names)
    else:
        # Single motif
        if '(' in motif_string:
            pattern_name = motif_string.split('(')[0]
            if pattern_name in tomtom_mapping:
                return tomtom_mapping[pattern_name]
            else:
                return pattern_name
        else:
            return motif_string

# Update columns
dev_library['TOMTOM_motifs'] = dev_library['TOMTOM_motifs'].apply(
    lambda x: replace_pattern_with_name(x, dev_tomtom_simple)
)

hk_library['TOMTOM_motifs'] = hk_library['TOMTOM_motifs'].apply(
    lambda x: replace_pattern_with_name(x, hk_tomtom_simple)
)

# Clean up any remaining 'pattern' references and standalone numbers
def clean_tomtom_motifs(motif_string):
    if motif_string == 'No motifs found':
        return 'No motifs found'
    
    # Remove 'pattern_' prefix
    cleaned = motif_string.replace('pattern_', '')
    
    # Remove standalone numbers (like "; 1" or "1;")
    import re
    # Remove semicolon followed by space and number
    cleaned = re.sub(r';\s*\d+', '', cleaned)
    # Remove number followed by semicolon
    cleaned = re.sub(r'\d+;\s*', '', cleaned)
    # Remove standalone numbers at the end
    cleaned = re.sub(r';\s*\d+$', '', cleaned)
    
    return cleaned

dev_library['TOMTOM_motifs'] = dev_library['TOMTOM_motifs'].apply(clean_tomtom_motifs)
hk_library['TOMTOM_motifs'] = hk_library['TOMTOM_motifs'].apply(clean_tomtom_motifs)

dev_library['motif_names'] = dev_library['TOMTOM_motifs'].apply(
    lambda x: extract_motif_names_only(x, dev_tomtom_simple)
)

hk_library['motif_names'] = hk_library['TOMTOM_motifs'].apply(
    lambda x: extract_motif_names_only(x, hk_tomtom_simple)
)

print(f"DEV TOMTOM motifs: {dev_tomtom_simple}")
print(f"HK TOMTOM motifs: {hk_tomtom_simple}")
print(f"DEV sequences with motifs: {len(dev_library[dev_library['motif_names'] != 'No motifs found'])}")
print(f"HK sequences with motifs: {len(hk_library[hk_library['motif_names'] != 'No motifs found'])}")

Sequences with multiple hits to pattern_0: {112: 2, 297: 2, 80: 2, 95: 2, 61: 2, 7: 3}
Sequences with multiple hits to pattern_1: {59: 2, 118: 2, 285: 2, 156: 2, 71: 2}
Sequences with multiple hits to pattern_2: {185: 2, 283: 2, 72: 2, 128: 2}
Sequences with multiple hits to pattern_0: {129: 2, 244: 2, 107: 2, 18: 2, 43: 2, 256: 2}
DEV TOMTOM motifs: {'pattern_0': 'MA0754.2', 'pattern_1': 'MA1141.1', 'pattern_2': 'MA0036.3'}
HK TOMTOM motifs: {'pattern_0': 'MA0755.1'}
DEV sequences with motifs: 117
HK sequences with motifs: 90


In [321]:
# Check what columns are available in the DataFrames
print("DEV Library columns:")
print(dev_library.columns.tolist())
print(f"\nDEV Library shape: {dev_library.shape}")

print("\nHK Library columns:")
print(hk_library.columns.tolist())
print(f"\nHK Library shape: {hk_library.shape}")


DEV Library columns:
['ce_name', 'orientation', 'ce_sequence', 'endogenous_sequence', 'pseudo_index', 'ce_start', 'ce_end', 'log2_fc', 'p_value', 'dataset', 'EvoAug_predictions', 'TOMTOM_motifs', 'motif_names']

DEV Library shape: (302, 13)

HK Library columns:
['ce_name', 'orientation', 'ce_sequence', 'endogenous_sequence', 'pseudo_index', 'ce_start', 'ce_end', 'log2_fc', 'p_value', 'dataset', 'EvoAug_predictions', 'TOMTOM_motifs', 'motif_names']

HK Library shape: (336, 13)


In [322]:
# Display top sequences by EvoAug predictions
dev_library.sort_values(by='EvoAug_predictions', ascending=False).head(5)


,ce_name,orientation,ce_sequence,endogenous_sequence,pseudo_index,ce_start,ce_end,log2_fc,p_value,dataset,EvoAug_predictions,TOMTOM_motifs,motif_names
281,"('Dref', 'GATAe', '+', '+', -5)",+,TATCGATAAT,TATATCTGTAGCTCAACACCTGACTGCTATCGAATGTTTTGGTATTTATTTTTATCGAAGTATTTTGCCGAAATTTGTGATTTGCCAGCCCTATTTCAATGTAAATATATCGATAATCGCCCGATTTTACTGAAAAATATTTCGATAAGAGCATCGATATTTCAAACAAATATCGACCCATTGACATGCGTCAGGTTTGTCCATCCCTAACAAAATCGTAGTACGAAATATCGAAAACGAAACACATCA,5585,107,116,4.574315,5.067241e-114,dev,5.307164,No motifs found,No motifs found
59,"('Jra', 'Jra', '-', '+', 10)",+,AGTGACTAATCTACTTACAAGTGAATCACG,AATCAGTTGAAATTTATACCTCATCGTCGACAATGCGAGTCGTCGCGTTCCCATTGTTTACAAAATGTACCATAAAGATTGCGAGCGAGAGAATTTGATTAGTGACTAATCTACTTACAAGTGAATCACGCGATCCAAAAATATTCATACAGAGCATTTTATCATTCGATTCGCTTTGCACCGAAACTCGGGACATTTCCACTGGGTGGGCAATCAATCAAGACCCATTTCGCCTAAACCTAAACATAT,2590,100,129,5.321928,1.763198e-07,dev,5.268528,"MA1141.1(104-154,fwd); MA1141.1(76-126,rev)","MA1141.1, MA1141.1"
167,"('Dref', 'Dref', '-', '+', 4)",+,TATCGATATCACTATCGATA,TTATTAGCCGGCAAAATTGGTGGGTAAACAACAAATATTTGGACAAAAGCGCAGACAAGAACTGATAAGAGGATCAGGGATGACGAATTTCAAATGGTTGCGCCAGAGATGACAGGGTATCGATATCACTATCGATAGACTTATCGAAGACGGCTTAGCACTTATACCTACTCTGCAGCCCTGGTCGGCAGAAAAACAATAATCAGCTGATGCCATCGCTTATTTCCAACAATTTGCACCGCAAAAGTG,19539,117,136,38.219281,4.230561e-34,dev,5.127045,"MA0754.2(104-154,rev)",MA0754.2
226,"('Jra', 'twi', '+', '-', 4)",+,GTGAATCACATCGACATATGT,GACTTTTCATTCCATTGTTCTTGGTTCTCGGAAGTATCAACTTATTGCCGAACTCGTGATTTAGTCTTATCGTTGTTTTTTTCTGGTTTTAGATATACATATGTTTTCTTTTATGCTAACTTCTGTCAGTGAATCACATCGACATATGTATTTGCATTGATATCTTGCGGATTTCCGAATGCCAGTAACGACGATTCACTGTACTTGGTTGGCGGAGCGCAATCTTTGTTCGAGAGAGTTCGTTTGCCA,626,128,148,34.219281,8.262587e-03,dev,5.113298,"MA1141.1(112-162,fwd)",MA1141.1
61,"('Dref', 'Max', '-', '+', 6)",+,TATCGATATTTGATAGCACGTG,TAATTAAGAGCAACAACAACAAACTTAGTAATGACACCCAAATAGAGATGAGCGATAGTTAATCACCTGTTATCAGCGCACAGCCAACGCTCATCCCTAAGCCTGCTGAGCAGCTGTTTCGACAATCGCTATCGATATATCTTGTTTAAACTTGCTATCGATATTTGATAGCACGTGTTGGGTTGTTCGCAATTTACGAATTGTATCTTTGTTAGTTATCCGCAATTTACATTTAAAACGAAGTAGATT,10613,155,176,35.219281,6.820836e-05,dev,5.038004,"MA0754.2(100-150,fwd); MA0754.2(142-192,rev)","MA0754.2, MA0754.2"


In [336]:
# Extract TF names from JASPAR database and update motif names
import re

def extract_tf_names_from_jaspar(jaspar_file):
    """Extract TF names from JASPAR database file."""
    tf_names = {}
    
    with open(jaspar_file, 'r') as f:
        content = f.read()
    
    # Find all MOTIF lines
    motif_lines = re.findall(r'MOTIF (MA\d+\.\d+) (MA\d+\.\d+\.(.+))', content)
    
    for ma_code, full_name, tf_name in motif_lines:
        tf_names[ma_code] = tf_name
    
    return tf_names

# Extract TF names from JASPAR database in CEseek_data folder
print("Extracting TF names from JASPAR database...")
jaspar_file_path = 'CEseek_data/JASPAR2022_CORE_vertebrates_non-redundant_pfms_meme.txt'
tf_names = extract_tf_names_from_jaspar(jaspar_file_path)

# Extract MA codes dynamically from our TOMTOM results
dev_ma_codes = set()
hk_ma_codes = set()

# Extract MA codes from DEV library motif names
for motif_name in dev_library['motif_names']:
    if motif_name != 'No motifs found':
        # Split by comma and semicolon to handle multiple motifs
        motifs = motif_name.replace(',', ';').split(';')
        for motif in motifs:
            motif = motif.strip()
            if motif and motif != 'No motifs found':
                dev_ma_codes.add(motif)

# Extract MA codes from HK library motif names
for motif_name in hk_library['motif_names']:
    if motif_name != 'No motifs found':
        # Split by comma and semicolon to handle multiple motifs
        motifs = motif_name.replace(',', ';').split(';')
        for motif in motifs:
            motif = motif.strip()
            if motif and motif != 'No motifs found':
                hk_ma_codes.add(motif)

# Combine all unique MA codes found
our_ma_codes = list(dev_ma_codes.union(hk_ma_codes))

print(f"\nDynamically found MA codes:")
print(f"  DEV library MA codes: {sorted(dev_ma_codes)}")
print(f"  HK library MA codes: {sorted(hk_ma_codes)}")
print(f"  All unique MA codes: {sorted(our_ma_codes)}")



def replace_ma_codes_with_tf_names(motif_string, tf_mapping):
    """Replace MA codes with actual TF names."""
    if motif_string == 'No motifs found':
        return 'No motifs found'
    
    # Split by semicolon and replace MA codes
    motifs = motif_string.split('; ')
    tf_names = []
    
    for motif in motifs:
        # Check if this motif contains an MA code
        for ma_code, tf_name in tf_mapping.items():
            if ma_code in motif:
                # Replace MA code with TF name
                motif = motif.replace(ma_code, tf_name)
                break
        tf_names.append(motif)
    
    # Remove duplicates and join
    unique_motifs = list(set(tf_names))
    return '; '.join(unique_motifs)

# Update motif_names column with actual TF names
print("\nUpdating motif names with actual TF names...")
dev_library['motif_names'] = dev_library['motif_names'].apply(
    lambda x: replace_ma_codes_with_tf_names(x, tf_names)
)

hk_library['motif_names'] = hk_library['motif_names'].apply(
    lambda x: replace_ma_codes_with_tf_names(x, tf_names)
)

# Display summary
print("\nDEV Library motif names summary (with TF names):")
dev_motif_names = dev_library[dev_library['motif_names'] != 'No motifs found']['motif_names']
print(f"Sequences with motifs: {len(dev_motif_names)}")
print(f"Unique motif names found: {set('; '.join(dev_motif_names).split('; '))}")

print("\nHK Library motif names summary (with TF names):")
hk_motif_names = hk_library[hk_library['motif_names'] != 'No motifs found']['motif_names']
print(f"Sequences with motifs: {len(hk_motif_names)}")
print(f"Unique motif names found: {set('; '.join(hk_motif_names).split('; '))}")


# Save updated DataFrames
dev_library.to_csv('Libraries/dev_modisco/dev_library_with_motifs.csv', index=True)
hk_library.to_csv('Libraries/hk_modisco/hk_library_with_motifs.csv', index=True)

print(f"\nUpdated DataFrames saved with actual TF names")

Extracting TF names from JASPAR database...

Dynamically found MA codes:
  DEV library MA codes: ['CUX1', 'FOS::JUND', 'GATA2']
  HK library MA codes: ['CUX2']
  All unique MA codes: ['CUX1', 'CUX2', 'FOS::JUND', 'GATA2']

Updating motif names with actual TF names...

DEV Library motif names summary (with TF names):
Sequences with motifs: 117
Unique motif names found: {'CUX1', 'CUX1, CUX1', 'FOS::JUND, GATA2, GATA2', 'CUX1, GATA2', 'FOS::JUND', 'CUX1, CUX1, CUX1', 'FOS::JUND, GATA2', 'FOS::JUND, FOS::JUND', 'GATA2', 'GATA2, GATA2'}

HK Library motif names summary (with TF names):
Sequences with motifs: 90
Unique motif names found: {'CUX2, CUX2', 'CUX2'}

Updated DataFrames saved with actual TF names


In [334]:
# Display top sequences by EvoAug predictions
dev_library.sort_values(by='EvoAug_predictions', ascending=False).head(5)

,ce_name,orientation,ce_sequence,endogenous_sequence,pseudo_index,ce_start,ce_end,log2_fc,p_value,dataset,EvoAug_predictions,TOMTOM_motifs,motif_names
281,"('Dref', 'GATAe', '+', '+', -5)",+,TATCGATAAT,TATATCTGTAGCTCAACACCTGACTGCTATCGAATGTTTTGGTATTTATTTTTATCGAAGTATTTTGCCGAAATTTGTGATTTGCCAGCCCTATTTCAATGTAAATATATCGATAATCGCCCGATTTTACTGAAAAATATTTCGATAAGAGCATCGATATTTCAAACAAATATCGACCCATTGACATGCGTCAGGTTTGTCCATCCCTAACAAAATCGTAGTACGAAATATCGAAAACGAAACACATCA,5585,107,116,4.574315,5.067241e-114,dev,5.307164,No motifs found,No motifs found
59,"('Jra', 'Jra', '-', '+', 10)",+,AGTGACTAATCTACTTACAAGTGAATCACG,AATCAGTTGAAATTTATACCTCATCGTCGACAATGCGAGTCGTCGCGTTCCCATTGTTTACAAAATGTACCATAAAGATTGCGAGCGAGAGAATTTGATTAGTGACTAATCTACTTACAAGTGAATCACGCGATCCAAAAATATTCATACAGAGCATTTTATCATTCGATTCGCTTTGCACCGAAACTCGGGACATTTCCACTGGGTGGGCAATCAATCAAGACCCATTTCGCCTAAACCTAAACATAT,2590,100,129,5.321928,1.763198e-07,dev,5.268528,"MA1141.1(104-154,fwd); MA1141.1(76-126,rev)","FOS::JUND, FOS::JUND"
167,"('Dref', 'Dref', '-', '+', 4)",+,TATCGATATCACTATCGATA,TTATTAGCCGGCAAAATTGGTGGGTAAACAACAAATATTTGGACAAAAGCGCAGACAAGAACTGATAAGAGGATCAGGGATGACGAATTTCAAATGGTTGCGCCAGAGATGACAGGGTATCGATATCACTATCGATAGACTTATCGAAGACGGCTTAGCACTTATACCTACTCTGCAGCCCTGGTCGGCAGAAAAACAATAATCAGCTGATGCCATCGCTTATTTCCAACAATTTGCACCGCAAAAGTG,19539,117,136,38.219281,4.230561e-34,dev,5.127045,"MA0754.2(104-154,rev)",CUX1
226,"('Jra', 'twi', '+', '-', 4)",+,GTGAATCACATCGACATATGT,GACTTTTCATTCCATTGTTCTTGGTTCTCGGAAGTATCAACTTATTGCCGAACTCGTGATTTAGTCTTATCGTTGTTTTTTTCTGGTTTTAGATATACATATGTTTTCTTTTATGCTAACTTCTGTCAGTGAATCACATCGACATATGTATTTGCATTGATATCTTGCGGATTTCCGAATGCCAGTAACGACGATTCACTGTACTTGGTTGGCGGAGCGCAATCTTTGTTCGAGAGAGTTCGTTTGCCA,626,128,148,34.219281,8.262587e-03,dev,5.113298,"MA1141.1(112-162,fwd)",FOS::JUND
61,"('Dref', 'Max', '-', '+', 6)",+,TATCGATATTTGATAGCACGTG,TAATTAAGAGCAACAACAACAAACTTAGTAATGACACCCAAATAGAGATGAGCGATAGTTAATCACCTGTTATCAGCGCACAGCCAACGCTCATCCCTAAGCCTGCTGAGCAGCTGTTTCGACAATCGCTATCGATATATCTTGTTTAAACTTGCTATCGATATTTGATAGCACGTGTTGGGTTGTTCGCAATTTACGAATTGTATCTTTGTTAGTTATCCGCAATTTACATTTAAAACGAAGTAGATT,10613,155,176,35.219281,6.820836e-05,dev,5.038004,"MA0754.2(100-150,fwd); MA0754.2(142-192,rev)","CUX1, CUX1"


In [335]:
# Check unique motif names
dev_library['motif_names'].unique()


array(['No motifs found', 'FOS::JUND', 'CUX1', 'CUX1, CUX1, CUX1',
       'GATA2', 'FOS::JUND, GATA2', 'FOS::JUND, FOS::JUND', 'CUX1, CUX1',
       'GATA2, GATA2', 'FOS::JUND, GATA2, GATA2', 'CUX1, GATA2'],
      dtype=object)

In [331]:
hk_library['motif_names'].unique()

array(['No motifs found', 'CUX2', 'CUX2, CUX2'], dtype=object)

In [332]:
# Display top HK sequences by EvoAug predictions
hk_library.sort_values(by='EvoAug_predictions', ascending=False).head(5)

,ce_name,orientation,ce_sequence,endogenous_sequence,pseudo_index,ce_start,ce_end,log2_fc,p_value,dataset,EvoAug_predictions,TOMTOM_motifs,motif_names
67,"('Dref', 'Max', '-', '+', 6)",+,TATCGATATTTGATAGCACGTG,TAATTAAGAGCAACAACAACAAACTTAGTAATGACACCCAAATAGAGATGAGCGATAGTTAATCACCTGTTATCAGCGCACAGCCAACGCTCATCCCTAAGCCTGCTGAGCAGCTGTTTCGACAATCGCTATCGATATATCTTGTTTAAACTTGCTATCGATATTTGATAGCACGTGTTGGGTTGTTCGCAATTTACGAATTGTATCTTTGTTAGTTATCCGCAATTTACATTTAAAACGAAGTAGATT,4637,155,176,35.219281,6.817735e-05,hk,7.520189,No motifs found,No motifs found
306,"('Dref', 'GATAe', '+', '+', -5)",+,TATCGATAAT,TATATCTGTAGCTCAACACCTGACTGCTATCGAATGTTTTGGTATTTATTTTTATCGAAGTATTTTGCCGAAATTTGTGATTTGCCAGCCCTATTTCAATGTAAATATATCGATAATCGCCCGATTTTACTGAAAAATATTTCGATAAGAGCATCGATATTTCAAACAAATATCGACCCATTGACATGCGTCAGGTTTGTCCATCCCTAACAAAATCGTAGTACGAAATATCGAAAACGAAACACATCA,605,107,116,6.174824,3.293969e-316,hk,7.510352,No motifs found,No motifs found
193,"('Dref', 'Dref', '-', '+', 4)",+,TATCGATATCACTATCGATA,TTATTAGCCGGCAAAATTGGTGGGTAAACAACAAATATTTGGACAAAAGCGCAGACAAGAACTGATAAGAGGATCAGGGATGACGAATTTCAAATGGTTGCGCCAGAGATGACAGGGTATCGATATCACTATCGATAGACTTATCGAAGACGGCTTAGCACTTATACCTACTCTGCAGCCCTGGTCGGCAGAAAAACAATAATCAGCTGATGCCATCGCTTATTTCCAACAATTTGCACCGCAAAAGTG,17547,117,136,38.678713,1.131588e-46,hk,7.450040,"MA0755.1(100-150,rev)",CUX2
125,"('Dref', 'Dref', '-', '+', 1)",+,ATTCGATACTATCGATA,TGAGAGTAATTTACCTTTTAAAACACTTGAAAAGGTTTTAAACACGTCGTTTTGCGAACGCGATTTATTAGAAAGGAAGATGGCCGTTTCAGAGATGGAAGAAATGGCGATGATTCGATACTATCGATATTTAAAGACTATCGATATTTTTCGTCCAACTGTTGTTGCCAGATCGTAGTGTATATTTGAATAATAAAAAATCTTAAGTAAGTTATGTTAATCTCAATACACTAAATTGTAAAATCACCT,133078,112,128,38.804243,7.311219e-51,hk,7.195486,"MA0755.1(108-158,fwd)",CUX2
72,"('Dref', 'Jra', '+', '+', -3)",+,TATCGATTACTCAGC,CGAGTTTAATTTAATTTCAGTTTCTGTTTCGACACGAGCAGTCAGCGAGAAGAATTTTGAAAAAATTACAGAGAGGGAAGCATTGCCAGTTGCTTTTATAGTACGGGCCGAGTCAGGGCTGCGGAGGCTCCGATAGAACTATCGATTACTCAGCTCGGGGCAATCGATGTAAACATCAGCTGTCATGTTCAATGTGGCAGCACCGGCCAACGCCGTTAATTAATTAGCTAATTCGAAAATGCTTAAATT,207805,139,153,5.321928,3.161250e-04,hk,7.171366,"MA0755.1(110-160,rev)",CUX2


In [ ]:
# Verify sequence lengths are consistent
len(dev_library[dev_library['motif_names'] == 'GATA2']['endogenous_sequence'].iloc[0]) == len(dev_library[dev_library['motif_names'] == 'GATA2']['endogenous_sequence'].iloc[1])


True

: 